In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests
# Import the API key
from config import g_key

In [2]:
# Store the CSV you saved in part one into a DataFrame
city_data_df = pd.read_csv('challenge_data/WeatherPy_challenge.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Ushuaia,AR,2020-04-19 05:53:19,-54.80,-68.30,44.60,75,40,16.11,scattered clouds,0.0,0.0
1,1,Hobart,AU,2020-04-19 05:53:21,-42.88,147.33,57.00,71,75,8.05,broken clouds,0.0,0.0
2,2,Rikitea,PF,2020-04-19 05:53:21,-23.12,-134.97,77.61,69,82,8.99,broken clouds,0.0,0.0
3,3,Vaini,TO,2020-04-19 05:51:02,-21.20,-175.20,78.80,78,40,10.29,smoke,0.0,0.0
4,4,Leningradskiy,RU,2020-04-19 05:50:38,69.38,178.42,4.48,89,20,7.56,few clouds,0.0,0.0


In [3]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [4]:
#city_data_df.dtypes

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_input = (input("Do you want it to be raining? (yes/no)"))
snow_input = (input("Do you want it to be snowing? (yes/no)"))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no)no
Do you want it to be snowing? (yes/no)no


In [7]:
# Filter the dataset to find the cities that fit the criteria
if rain_input == "no" and snow_input == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Rain inches (last 3 hrs)"] == 0) & (city_data_df["Snow inches (last 3 hrs)"] == 0)]
elif rain_input == "yes" and snow_input == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Rain inches (last 3 hrs)"] > 0) & (city_data_df["Snow inches (last 3 hrs)"] > 0)]
elif rain_input == "yes" and snow_input == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Rain inches (last 3 hrs)"] > 0) & (city_data_df["Snow inches (last 3 hrs)"] == 0)]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Rain inches (last 3 hrs)"] == 0) & (city_data_df["Snow inches (last 3 hrs)"] > 0)]
    

In [8]:
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
2,2,Rikitea,PF,2020-04-19 05:53:21,-23.12,-134.97,77.61,69,82,8.99,broken clouds,0.0,0.0
3,3,Vaini,TO,2020-04-19 05:51:02,-21.20,-175.20,78.80,78,40,10.29,smoke,0.0,0.0
15,15,Salinopolis,BR,2020-04-19 05:53:23,-0.61,-47.36,80.06,80,98,4.97,overcast clouds,0.0,0.0
16,16,Kupang,ID,2020-04-19 05:51:49,-10.17,123.58,89.60,70,40,9.17,scattered clouds,0.0,0.0
19,19,Pandan,PH,2020-04-19 05:53:24,14.05,124.17,84.60,72,84,11.25,broken clouds,0.0,0.0


In [9]:
preferred_cities_df.count()

City_ID                     99
City                        99
Country                     98
Date                        99
Lat                         99
Lng                         99
Max Temp                    99
Humidity                    99
Cloudiness                  99
Wind Speed                  99
Current Description         99
Rain inches (last 3 hrs)    99
Snow inches (last 3 hrs)    99
dtype: int64

In [10]:
# Create a DataFrame to store Hotel names
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [11]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    #Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the lat and long to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our lat and long
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,77.61,broken clouds,-23.12,-134.97,Pension Maro'i
3,Vaini,TO,78.80,smoke,-21.20,-175.20,Keleti Beach Resort
15,Salinopolis,BR,80.06,overcast clouds,-0.61,-47.36,Hotel Salinópolis
16,Kupang,ID,89.60,scattered clouds,-10.17,123.58,Sylvia Hotel Premier Kupang
19,Pandan,PH,84.60,broken clouds,14.05,124.17,Bonifacio Lodge


In [13]:
# Save the hotel df as csv file
output_data_file = "challenge_data/WeatherPy_vacation.csv"
# Export hotel_df into a CSV
hotel_df.to_csv(output_data_file)

In [14]:
# Create template for info box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))